In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import optimizers

In [ ]:
train_path = r"FER-2013\train"
test_path = r"FER-2013\test"

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,               # Normalize pixel values from [0, 255] to [0, 1]
                                    rotation_range=15,            # Randomly rotate images in the range of ±15 degrees
                                    width_shift_range=0.1,        # Randomly shift images horizontally by up to 10% of the width
                                    height_shift_range=0.1,       # Randomly shift images vertically by up to 10% of the height
                                    shear_range=0.1,              # Apply shear transformations (slanting the image)
                                    zoom_range=0.1,               # Randomly zoom in/out by up to 10%
                                    horizontal_flip=True,         # Randomly flip images horizontally (left-right)
                                    fill_mode='nearest'           # Fill in missing pixels after transformation using the nearest pixel values
                                    )

test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# Exemplary images
x_batch, y_batch = next(train_set)
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(x_batch[i])
    plt.title(f"Class: {np.argmax(y_batch[i])}")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Adding class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_set.classes),
    y=train_set.classes
)
class_weights = dict(enumerate(class_weights))

In [ ]:
# Base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

#Option 1: Freeze all layers //turned out to give lower accuracy
#base_model.trainable = False  # Freeze for transfer learning

#Option 2: Freeze all layers except the last 30 layers
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(train_set.num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=output)

print(model.summary())

In [ ]:
# Callback that changes learning rate when model has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Compile the model
model.compile(optimizer=Adam(learning_rate = 0.0001, 
                          beta_1 = 0.9, beta_2 = 0.999),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

#training
history = model.fit(
    train_set,
    steps_per_epoch=len(train_set),
    epochs=40,
    validation_data=test_set,
    validation_steps=len(test_set),
    callbacks=[reduce_lr],
    class_weight=class_weights
)

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(accuracy) + 1) # get the number of epochs for X axsis

plt.figure(figsize=(12, 6))
plt.plot(epochs, accuracy, color='blue', label='Training accuracy')
plt.plot(epochs, val_accuracy, color='orange', label='Validation accuracy')
plt.title('Training vs validation accuracy')
plt.grid(True)
plt.legend()

plt.figure(figsize=(12, 6))
plt.plot(epochs, loss, color='blue', label='Training loss')
plt.plot(epochs, val_loss, color='orange', label='Validation loss')
plt.title('Training vs validation loss')
plt.grid(True)
plt.legend()

plt.show()

In [ ]:
Y_pred = model.predict(test_set)
y_pred = np.argmax(Y_pred, axis=1)

print(classification_report(test_set.classes, y_pred))